In [1]:
pip install sympy

   ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.7 MB 653.6 kB/s eta 0:00:09
   - -------------------------------------- 0.2/5.7 MB 1.7 MB/s eta 0:00:04
   -- ------------------------------------- 0.3/5.7 MB 2.2 MB/s eta 0:00:03
   ---- ----------------------------------- 0.6/5.7 MB 3.2 MB/s eta 0:00:02
   ----- ---------------------------------- 0.9/5.7 MB 3.6 MB/s eta 0:00:02
   ------- -------------------------------- 1.1/5.7 MB 3.8 MB/s eta 0:00:02
   --------- ------------------------------ 1.3/5.7 MB 4.0 MB/s eta 0:00:02
   ----------- ---------------------------- 1.6/5.7 MB 4.2 MB/s eta 0:00:01
   ------------ --------------------------- 1.8/5.7 MB 4.3 MB/s eta 0:00:01
   -------------- ------------------------- 2.1/5.7 MB 4.5 MB/s eta 0:00:01
   ----------------- ---------------------- 2.4/5.7 MB 4.6 MB/s eta 0:00:01
   ------------------- -------------------- 2.7/5.7 MB 4.8 MB/s eta 0:00:01
   --------------

In [4]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install minikanren

     ---------------------------------------- 0.0/41.3 kB ? eta -:--:--
     -------------------------------------  41.0/41.3 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 41.3/41.3 kB 501.5 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/56.1 kB ? eta -:--:--
   ---------------------------------------- 56.1/56.1 kB 1.4 MB/s eta 0:00:00
  Created wheel for minikanren: filename=miniKanren-1.0.3-py3-none-any.whl size=23948 sha256=834d263c91358fdbbf30e99a8da2ae5343672d93d027d59a7ca9ec78b84574a7
  Stored in directory: c:\users\admin\appdata\local\pi

Exercise 1

In [6]:
import json
from kanren import Relation, facts, run, conde, var, eq

In [7]:
# Check if 'x' is the parent of 'y'
def parent(x, y):
    return conde([father(x,y)], [mother(x,y)])

# Check if 'x' is the grandparent of 'y'
def grandparent(x, y):
    temp = var()
    return conde((parent(x,temp), parent(temp, y)))

# Check for sibling relationship between 'a' and 'b'
def sibling(x, y):
    temp = var()
    return conde((parent(temp, x), parent(temp, y)))

# Check if x is y's uncle
def uncle(x, y):
    temp = var()
    return conde((father(temp, x), grandparent(temp, y)))

In [9]:
# print("__name__ : ", __name__)
if __name__ == '__main__':
    father = Relation() # creating an instance of Relation (instantiating)
    mother = Relation()
    
    with open('relationships.json', 'r') as f:
        d=json.loads(f.read())
        
    for item in d['father']:
        facts(father, (list(item.keys())[0],list(item.values())[0]))
      
    for item in d['mother']:
        facts(mother, (list(item.keys())[0],list(item.values())[0]))

    x = var()

In [10]:
# John's children
name = 'John'
output = run(0, x, father(name, x))

print(name + 'children are : ')

for item in output:
    print(item)


Johnchildren are : 
David
William
Adam


In [11]:
# Adam's parents
name = 'Adam'
output = run(0, x, parent(name, x))

print(name + 'parents are: ')

for item in output:
    print(item)

Adamparents are: 
Sophia


In [13]:
# Megan's grandchildren
name = 'Megan'
output = run(0, x, grandparent(name, x))

print(name + 'fc are : ', output)

Meganfc are :  ('Sophia', 'Wayne', 'Chris', 'Neil', 'Stephanie', 'Tiffany', 'Julie', 'Peter')


In [14]:
# David's siblings
name = 'David'
output = run(0, x, sibling(name, x))

print(name + 'siblings are: ')
for item in output:
    print(item)


Davidsiblings are: 
David
David
Adam
Adam
William
William


In [15]:
# Tiffany's uncles
name = 'Tiffany'
output = run(0, x, uncle(x, name))
father_name = run(1, x, father(x, name))
print(father_name[0])
[n for n in output if n!=father_name[0]]

David


['Adam', 'William']

In [16]:
# all spouses
a, b, c = var(), var(), var()
output = run(0, (a, b), father(a, c), mother(b, c))
output

(('Adam', 'Lily'),
 ('William', 'Emma'),
 ('John', 'Megan'),
 ('David', 'Olivia'),
 ('John', 'Megan'),
 ('William', 'Emma'),
 ('David', 'Olivia'),
 ('John', 'Megan'),
 ('David', 'Olivia'),
 ('David', 'Olivia'))

Exercise 2

In [17]:
import json 
from kanren import run, fact, eq, Relation, var, conde

In [18]:
from kanren import run, fact, eq, Relation, var

adjacent = Relation()
coastal = Relation()

In [26]:
file_coastal = 'coastal_states.txt'
file_adjacent ='adjacent_states.txt'

In [27]:
# Read the file containing the coastal states
with open(file_coastal, 'r') as f:
    line = f.read()
    coastal_states = line.split()

# Add the info to the fact base
for state in coastal_states:
    fact(coastal, state)

# Read the file containing the coastal states
with open(file_adjacent, 'r') as f:
    adjlist = [line.strip().split() for line in f if line and line[0].isalpha()]

# Add the info to the fact base
for L in adjlist:
    head, tail = L[0], L[1:]
    for state in tail:
        fact(adjacent, head, state)

In [28]:
# Initialize the variables
x = var()
y = var()

#Example 1
# Is Nevada adjacent to Louisiana?
output = run(0, x, adjacent('Nevada', 'Louisiana'))
print('\nIs Nevada adjacent to Louisiana?:')
print('Yes' if len(output) else 'No')

In [29]:
#Example 2
# States adjacent to Oregon
output = run(0, x, adjacent('Oregon', x))
print('\nList of states adjacent to Oregon:')
for item in output:
    print(item)


List of states adjacent to Oregon:


In [31]:
#Example 3
# States adjacent to Mississippi that are coastal
output = run(0, x, adjacent('oregon', x), coastal(x))
print('\nList of states adjacent to Mississippi:')
for item in output:
    print(item)


List of states adjacent to Mississippi:


In [33]:
#Example of 'n' states that border a coastal state

n=7
output = run(n, x, coastal(y), adjacent(x,y))
print('\nList of ' +str(n) + ' states that border a coastal state: ')
for item in output:
    print(item)


List of 7 states that border a coastal state: 
Maine,New


In [34]:
#list of states that adjacent to the two given states
output = run(0, x, adjacent('Arkansas', x),adjacent('Kentucky', x))
print('\nList of states that are adjacent to Arkansas and Kentucky:')
for item in output:
    print(item)


List of states that are adjacent to Arkansas and Kentucky:
